# Advent of Code 2020 - Day 5

In [188]:
interface Partition {
    val id: Char
    val half: Half

    enum class Half {
        UPPER,
        LOWER,
    }
}

enum class RowPartition(override val id: Char, override val half: Partition.Half) : Partition {
    FRONT('F', Partition.Half.UPPER),
    BACK('B', Partition.Half.LOWER),
    ;
    companion object {
        private val idMap: Map<Char, RowPartition> = RowPartition.values().associateBy(RowPartition::id)
        fun fromChar(char: Char): RowPartition = idMap.getValue(char)
    }
}

enum class ColumnPartition(override val id: Char, override val half: Partition.Half) : Partition {
    LEFT('L', Partition.Half.UPPER),
    RIGHT('R', Partition.Half.LOWER),
    ;
    companion object {
        private val idMap: Map<Char, ColumnPartition> = ColumnPartition.values().associateBy(ColumnPartition::id)
        fun fromChar(char: Char): ColumnPartition = idMap.getValue(char)
    }
}

data class Seat(val rowPartitions: List<RowPartition>, val columnPartitions: List<ColumnPartition>)

In [189]:
import java.io.File

val seats: List<Seat> = File("Day5.input.txt")
    .bufferedReader()
    .readLines()
    .map { line -> line.slice(0..6) to line.slice(7..9) }
    .map { (row, col) -> Seat(row.map(RowPartition::fromChar), col.map(ColumnPartition::fromChar)) }

## Part 1

Use binary space partitioning to find each `Seat`'s unique ID. A `FRONT` partition means that the seat is in the `UPPER` half of the plane and `BACK` is in the `LOWER` half. Likewise, `LEFT` is in the `UPPER` half and `RIGHT` is in the `LOWER` half. Each `Partition` subsequently breaks up the plane into further halves until a single seat is left over.

The plane has 128 rows and 8 columns. To find a `Seat`'s unique ID, multiply its row by 8 and add the value to its column.

In [190]:
infix fun Int.pow(exponent: Int): Int = toDouble().pow(exponent).toInt()

fun List<Partition>.id(): Int =
    fold(0..(2 pow count())) { range, partition ->
        when (partition.half) {
            Partition.Half.UPPER -> (range.start)..((range.start + range.endInclusive) / 2)
            Partition.Half.LOWER -> ((range.start + range.endInclusive) / 2)..(range.endInclusive)
        }
    }
    .start

seats
    .map { it.rowPartitions.id() * 8 + it.columnPartitions.id() }
    .maxOrNull()

885

In [191]:
fun List<Partition>.cleverId(): Int =
    joinToString("") {
        when (it.half) {
            Partition.Half.UPPER -> "0"
            Partition.Half.LOWER -> "1"
        }
     }
     .toInt(2)

val Seat.id: Int
     get() = rowPartitions.cleverId() * 8 + columnPartitions.cleverId()

seats
    .map { it.id }
    .maxOrNull()

885

### Notes

A binary search problem, but after finishing the problem I wanted to see if there was a more clever way to do it with bit-shifting. The "a-ha" moment came after realizing `UPPER` and `LOWER` just represented a bit and simply treating the string as a binary would give you the ID. Felt pretty good about this one.

`String.toInt` takes in a radix which makes parsing a binary's string presentation easy.

## Part 2

Find the missing seat. The seats at the very front and very back do not exist.

In [192]:
fun Iterable<Int>.xorAll() = reduce { acc, value -> value xor acc }

seats
    .map { it.id }
    .let { 
        val min = it.minOrNull()!!
        val max = it.maxOrNull()!!
        it.xorAll() xor (min..max).xorAll()
     }

623

### Notes

I knew that this one was an XOR problem and I basically needed a bitmask but got tripped up in a couple of places.

Firstly, the problem was a bit unclear (should ask for clarification!), I thought it meant the first and last seat did not exist (two seats in total). Rather, _seats_ in the front and back were missing - so it was just a subsection in the middle.

Secondly, I got tripped up on what I was XORing. Clearly I needed to XOR my input, but that was not enough. To find the missing seat, you also need to XOR the contiguous range your seat is in. That way when you XOR both of them together, the missing seat is distinguished.

The "simpler" (or rather, without bitwise) would be to just sort the values and iterate over them to see if the N+1 element was missing.